To display images in notebook

In [1]:
%matplotlib inline

Loading our tools and libraries

In [2]:
import numpy as np
import ipyvolume as ipv
import matplotlib.pyplot as plt
import pathlib, glob

In [3]:
import sys
sys.path.append('../src/')
from molecular_handling import particle
from molecular_handling import generate_dataset
from scipy.spatial.transform import Rotation as R
from forward_modeling import project_volume

Make sure we have a local directory to write the data in:

In [4]:
data_directory = pathlib.Path('..') / 'data'  # directory where the data is
output_dir = data_directory / 'test'
output_dir.mkdir(parents=True, exist_ok=True)

# Single particle dataset

Build a particle with 3 atoms of radii 1, 2 and 1.

In [5]:
crds = np.array([[0.,5.,5.],[5.,5.,5.],[5.,5.,0.]])
particle_ = particle(n_atom=3, rads_atom=[2.,2.,2.], crds = crds)

Generate a dataset from that particle and save it:

In [6]:
data, metadata = generate_dataset(particle_, n_projections=10000, reldir=output_dir, print_frequency=1000)

1000 images were created
2000 images were created
3000 images were created
4000 images were created
5000 images were created
6000 images were created
7000 images were created
8000 images were created
9000 images were created
10000 images were created


In [7]:
crds = np.array([[0.,5.,5.],[5.,5.,5.],[5.,5.,0.]])
particle_ = particle(n_atom=3, rads_atom=[2.,2.,2.], crds = crds, size_grid=33)
data, metadata = generate_dataset(particle_, n_projections=10000,keyword='particle_oddres', reldir=output_dir, print_frequency=1000)

1000 images were created
2000 images were created
3000 images were created
4000 images were created
5000 images were created
6000 images were created
7000 images were created
8000 images were created
9000 images were created
10000 images were created


Higher "resolution":

In [8]:
crds = np.array([[4.,7.,7.],[5.,3.,6.],[6.,6.,6.]])
particle_ = particle(n_atom=3, rads_atom=[2.,2.,2.], crds = crds,size_grid=128)
data, metadata = generate_dataset(particle_, n_projections=100, keyword='particle_hires', reldir=output_dir)

100 images were created


Let's see what we have just written:

In [9]:
glob.glob(f'{output_dir}/*')

['../data/test/particle_hires_xyz.npy',
 '../data/test/particle_oddres_meta.npy',
 '../data/test/particle_hires_map.npy',
 '../data/test/particle_meta.npy',
 '../data/test/het_particle_meta.npy',
 '../data/test/particle_hires_data.npy',
 '../data/test/particle_hires_meta.npy',
 '../data/test/particle_map.npy',
 '../data/test/particle_oddres_data.npy',
 '../data/test/particle_xyz.npy',
 '../data/test/particle_oddres_map.npy',
 '../data/test/het_particle_data.npy',
 '../data/test/particle_oddres_xyz.npy',
 '../data/test/het_particle_map.npy',
 '../data/test/particle_data.npy']

# Heterogeneous dataset

In [10]:
#building the conformations
crds1 = np.array([[1.,7.,9.],[7.,7.,7.],[9.,7.,1.]])
particle1 = particle(n_atom=3, rads_atom=[2.,3.,2.], crds = crds1, size_grid=33)
particle1.create_map()
crds2 = np.array([[1.,7.,7.],[7.,7.,7.],[7.,7.,1.]])
particle2 = particle(n_atom=3, rads_atom=[2.,3., 2.], crds = crds2, size_grid=33)
particle2.create_map()
crds3 = np.array([[1.,7.,5.],[7.,7.,7.],[5.,7.,1.]])
particle3 = particle(n_atom=3, rads_atom=[2.,3., 2.], crds = crds3, size_grid=33)
particle3.create_map()
import ipyvolume as ipv
vols = np.array([particle1.volume, particle2.volume, particle3.volume])

In [11]:
ipv.quickvolshow(vols[0], level=[0.25, 0.75], opacity=[0.03, 0.2], level_width=0.1)

/Users/michaelresplandy/miniconda3/envs/gmm-cryoem/lib/python3.6/site-packages/ipyvolume/serialize.py:92: RuntimeWarning: invalid value encountered in true_divide
  gradient = gradient / np.sqrt(gradient[0] ** 2 + gradient[1] ** 2 + gradient[2] ** 2)


In [12]:
ipv.quickvolshow(vols[1], level=[0.25, 0.75], opacity=[0.03, 0.2], level_width=0.1)

In [13]:
ipv.quickvolshow(vols[2], level=[0.25, 0.75], opacity=[0.03, 0.2], level_width=0.1)

In [14]:
#building the covariance of each volume
covs = np.array([np.identity(np.power(particle1.size_grid, 3))/50 for i in range(len(vols))])
#cholesky decomposition for sample generation
chol_covs = [np.linalg.cholesky(cov) for cov in covs]
#mixture parameter
pis = [0.2, 0.3, 0.5]

In [15]:
def sample(mean, chol_cov):
    return mean + chol_cov @ np.random.standard_normal(mean.size)

def generate_dataset(vols, chol_covs, pis, size_dataset = 100, output_maps = True, reldir='.', keyword='het_particle', heterogeneity = True, print_frequency = 50):
    N = vols.shape[-1]
    pis = np.cumsum(pis)
    dataset = np.zeros((size_dataset, N, N))
    metadataset = np.zeros((size_dataset, 6))
    
    if output_maps: #needs to be called before rotating the particle
        np.save(f'{reldir}/{keyword}_map', vols)
    
    if heterogeneity:
        vols = vols.reshape(vols.shape[0], np.power(N, 3)
                    )
        
    for i in range(len(dataset)):
        p = np.random.random()
        index = len(pis) - pis[pis-p>0].shape[0]
        mean = vols[index]
        chol_cov = chol_covs[index]
        vol_sample = sample(mean, chol_cov).reshape((N,N,N))
        
        rotation =  R.random()
        image = project_volume(vol_sample, rotation)
        
        dataset[i,...] = image
        metadataset[i,0:3] = -np.array(rotation.as_rotvec())
        metadataset[i,3] = N
        metadataset[i,4] = len(vols)
        metadataset[i,5] = index
        
        
        if (i+1)%print_frequency == 0:
            print(f'{str(i+1)} images were created')
    
    np.save(f'{reldir}/{keyword}_data', dataset) 
    np.save(f'{reldir}/{keyword}_meta', metadataset) 
    
    return dataset, metadataset


In [16]:
data, metadata = generate_dataset(vols, covs, pis, size_dataset = 300, reldir = output_dir)

50 images were created
100 images were created
150 images were created
200 images were created
250 images were created
300 images were created
